In [1]:
# python notebook for Make Your Own Neural Network
# xytywh, 2019/06/18

In [2]:
import numpy as np
# scipy.special for the sigmoid function expit()
import scipy.special

In [3]:
# neural network class definition
# 神经网络类定义
class neuralNetwork:
    
    
    # initialise the neural network
    # 初始化神经网络
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        # 设置每一个输入层、隐藏层、输出层的结点个数
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        # 在输入层与隐藏层之间的链接权重矩阵Wih，大小为hnodes乘以inodes
        # 在隐藏层和输出层之间的链接权重矩阵Who，大小为hnodes乘以onodes
        
        # self.wih = (np.random.rand(self.hnodes, self.inodes) - 0.5)
        # self.who = (np.random.rand(self.onodes, self.hnodes) - 0.5)
        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        #学习率
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        # 将维度为(3,)的一维数据，变成维度为(1,3)的二维数据，再转置，变成维度为(3,1)的二维数据
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        
        
        
        # 前向传播(feedforward)
        
        # calculate signals into hidden layer
        # 计算隐藏层的输入
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        # 计算隐藏层的输出
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        # 计算最后输出层的输入
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        # 计算最后输出层的输出
        # 这里使用sigmoid函数不太对，对于多分类问题，并且0-9每一类是互斥的，其实应该使用softmax，
        # 不过为了方便起见，就使用sigmoid，不会有太大影响
        final_outputs = self.activation_function(final_inputs)
        print("\nhidden_outputs shape is:{}\nfinal_outputs shape is:{}".format(hidden_outputs.shape,final_outputs.shape))
        
        
        
        
        # 反向传播(backprop)
        
        # output layer error is the (target - actual)
        # 输出层误差为(targets-final_outputs)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = np.dot(self.who.T, output_errors) 
        print("hidden_errors shape is:{}".format(hidden_errors.shape))
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        pass

    
    # query the neural network
    # 查询神经网络，接受输入，返回输出
    def query(self, inputs_list):
        # convert inputs list to 2d array
        # 将维度为(3,)的一维数据，变成维度为(1,3)的二维数据
        inputs = np.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [4]:
# number of input, hidden and output nodes
# 输入，隐藏，输出层结点个数
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# learning rate is 0.3
learning_rate = 0.3

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [5]:
# test query (doesn't mean anything useful yet)
n.query([1.0, 0.5, -1.5])

array([[0.37815053],
       [0.53051953],
       [0.38520763]])

In [6]:
# 维度为(2,3)，每一行代表一个数据，一共两个数据
n.query([[1.0, 0.5, -1.5],[1.0,0.5,-1.5]])

array([[0.37815053, 0.37815053],
       [0.53051953, 0.53051953],
       [0.38520763, 0.38520763]])

In [7]:
# 维度为(3,3)，每一行代表一个数据，一共三个数据
n.query([[1.0, 0.5, -1.5],[1.0,0.5,-1.5],[1.0,2.0,3.0]])

array([[0.37815053, 0.37815053, 0.34952995],
       [0.53051953, 0.53051953, 0.37573375],
       [0.38520763, 0.38520763, 0.47477568]])

In [8]:
input_list = [[0.1,0.5,0.4],[0.3,0.4,0.3]]
target_list = [[1.0,0,0],[1.0,0,0]]
print("before train:\n")
print("wih{}\n\nwho{}".format(n.wih,n.who))
n.train(input_list,target_list)
print("\nafter train once:\n")
print("wih{}\n\nwho{}".format(n.wih,n.who))

before train:

wih[[ 0.21035666 -0.05310695 -0.682246  ]
 [ 0.55508667  0.74618038  0.39225083]
 [ 0.21373902  0.37047822 -0.06060418]]

who[[ 0.08457183 -0.03493322 -0.874248  ]
 [ 0.49910377 -0.88645364  0.4125856 ]
 [-0.6707314  -0.22307466  0.2888873 ]]

hidden_outputs shape is:(3, 2)
final_outputs shape is:(3, 2)
hidden_errors shape is:(3, 2)

after train once:

wih[[ 0.21345556 -0.04606043 -0.67676239]
 [ 0.56864207  0.77662432  0.4159272 ]
 [ 0.18833664  0.31331145 -0.10506769]]

who[[ 0.12342827  0.02111098 -0.82650411]
 [ 0.4678531  -0.9315208   0.37419244]
 [-0.69883417 -0.26361695  0.25435014]]
